In [176]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [177]:

df = pd.read_csv('Train_ee.csv')
dft=pd.read_csv('Test_ee.csv')

In [178]:
df

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date,Selling_Price
0,P-2610,B-659,bags wallets belts,bags,hand bags,4.3,2/3/2017,291.0
1,P-2453,B-3078,clothing,women s clothing,western wear,3.1,7/1/2015,897.0
2,P-6802,B-1810,home decor festive needs,showpieces,ethnic,3.5,1/12/2019,792.0
3,P-4452,B-3078,beauty and personal care,eye care,h2o plus eye care,4.0,12/12/2014,837.0
4,P-8454,B-3078,clothing,men s clothing,t shirts,4.3,12/12/2013,470.0
...,...,...,...,...,...,...,...,...
2447,P-8870,B-2292,clothing,kids clothing,girls wear,2.3,11/3/2017,741.0
2448,P-10672,B-3078,footwear,women s footwear,casual shoes,1.9,4/12/2016,1590.0
2449,P-2134,B-479,clothing,men s clothing,t shirts,1.9,19/4/2019,995.0
2450,P-724,B-133,automotive,accessories spare parts,car interior exterior,2.7,1/12/2014,1598.0


In [179]:
dft

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date
0,P-11284,B-2984,computers,network components,routers,4.3,1/12/2018
1,P-6580,B-1732,jewellery,bangles bracelets armlets,bracelets,3.0,20/12/2012
2,P-5843,B-3078,clothing,women s clothing,western wear,1.5,1/12/2014
3,P-5334,B-1421,jewellery,necklaces chains,necklaces,3.9,1/12/2019
4,P-5586,B-3078,clothing,women s clothing,western wear,1.4,1/12/2017
...,...,...,...,...,...,...,...
1046,P-9758,B-2543,sports fitness,outdoor adventure,camping hiking,2.0,28/2/2014
1047,P-11898,B-3197,jewellery,necklaces chains,necklaces,4.7,1/12/2019
1048,P-904,B-133,automotive,accessories spare parts,car interior exterior,3.5,1/12/2014
1049,P-1714,B-358,kitchen dining,lighting,bulbs,2.4,23/6/2013


In [180]:
dft.Product_Brand.nunique()

494

In [181]:
df.Item_Category.nunique()

75

In [182]:
df.Subcategory_1.nunique()

130

In [183]:
df.Subcategory_2.nunique()

336

In [184]:
final_df=pd.concat([df,dft],axis=0)

In [185]:
final_df.drop("Selling_Price",inplace=True,axis=1)

In [186]:
final_df

,Date,Item_Category,Item_Rating,Product,Product_Brand,Subcategory_1,Subcategory_2
0,2/3/2017,bags wallets belts,4.3,P-2610,B-659,bags,hand bags
1,7/1/2015,clothing,3.1,P-2453,B-3078,women s clothing,western wear
2,1/12/2019,home decor festive needs,3.5,P-6802,B-1810,showpieces,ethnic
3,12/12/2014,beauty and personal care,4.0,P-4452,B-3078,eye care,h2o plus eye care
4,12/12/2013,clothing,4.3,P-8454,B-3078,men s clothing,t shirts
...,...,...,...,...,...,...,...
1046,28/2/2014,sports fitness,2.0,P-9758,B-2543,outdoor adventure,camping hiking
1047,1/12/2019,jewellery,4.7,P-11898,B-3197,necklaces chains,necklaces
1048,1/12/2014,automotive,3.5,P-904,B-133,accessories spare parts,car interior exterior
1049,23/6/2013,kitchen dining,2.4,P-1714,B-358,lighting,bulbs


In [187]:
final_df['Date'] = pd.to_datetime(final_df['Date'])

In [188]:

final_df['start_Y'] = final_df['Date'].apply(lambda x: x.year)
final_df['start_M'] = final_df['Date'].apply(lambda x: x.month)
final_df['start_D'] = final_df['Date'].apply(lambda x: x.day)
final_df.drop("Date",inplace=True,axis=1)

In [189]:
final_df.drop("Product",inplace=True,axis=1)

In [190]:
def category_onehot_multcols(multcolumns):
    df_final=final_df
    i=0
    for fields in multcolumns:
        
        print(fields)
        df1=pd.get_dummies(final_df[fields],drop_first=True)
        
        final_df.drop([fields],axis=1,inplace=True)
        if i==0:
            df_final=df1.copy()
        else:
            
            df_final=pd.concat([df_final,df1],axis=1)
        i=i+1
       
        
    df_final=pd.concat([final_df,df_final],axis=1)
        
    return df_final

In [191]:
columns=["Product","Item_Category","Product_Brand","Subcategory_1","Subcategory_2"]
final_df=category_onehot_multcols(columns)

Product
Item_Category
Product_Brand
Subcategory_1
Subcategory_2
start_Y
start_M
start_D


In [192]:
final_df

,Item_Rating,P-10003,P-10005,P-10006,P-10010,P-10012,P-10016,P-10018,P-10019,P-10025,...,22,23,24,25,26,27,28,29,30,31
0,4.3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3.5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1047,4.7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1048,3.5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1049,2.4,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [197]:
x=final_df.iloc[:2452,:]
x1=final_df.iloc[2452:,:]

In [198]:
x["target"]=df["Selling_Price"]

In [199]:
train=x
test=x1

In [200]:
TARGET_COL = "target"
features = [c for c in train.columns if c not in [TARGET_COL]]
target = train[TARGET_COL]

In [201]:

def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

In [202]:
print(len(features))

5487


In [203]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import *
import time
training_start_time = time.time()

max_iter = 10
folds = StratifiedKFold(n_splits = max_iter)
oofs = np.zeros(len(train))
test_preds = np.zeros(len(test))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, pd.qcut(target, 10, labels=False, duplicates='drop'))):
    
    print(f'\n---- Fold {fold_} -----\n')
    
    fold_start_time = time.time()
    
    X_trn, y_trn = train.iloc[trn_idx][features], target.iloc[trn_idx]
    X_val, y_val = train.iloc[val_idx][features], target.iloc[val_idx]
    X_test = test[features]
    print(X_trn.shape[1], X_val.shape[1])
    
    
    clf = LGBMRegressor(n_estimators=1000, num_leaves=127, max_depth=-1,min_child_samples=4, learning_rate=0.02, colsample_bytree=0.4, reg_alpha=0.5, reg_lambda=2)
    _ = clf.fit(X_trn, np.log(y_trn), eval_set = [(X_val, np.log(y_val))], verbose=100, early_stopping_rounds=200, eval_metric='rmse')

    oofs[val_idx] = np.exp(clf.predict(X_val))
    current_test_pred = np.exp(clf.predict(X_test))
    test_preds += np.exp(clf.predict(X_test))/max_iter
    
    
    print(f'\n Fold {rmse(np.log(y_val), np.log(oofs[val_idx]))}')
    
    fold_end_time = time.time()
    total_fold_time = int(fold_end_time - fold_start_time)
    
    print(f"\n->-> Fold ran for {(total_fold_time)//60} minutes {(total_fold_time)%60} seconds")
    

print(f'\nOOF val score: {rmse(np.log(target), np.log(oofs))}')
training_end_time = time.time()
total_training_time = int(training_end_time - training_start_time)

print(f'\n->-> Total training time: {(total_training_time)//60} minutes {(total_training_time)%60} seconds')


---- Fold 0 -----

5509 5509
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.697585	valid_0's l2: 0.486625
[200]	valid_0's rmse: 0.658964	valid_0's l2: 0.434234
[300]	valid_0's rmse: 0.658891	valid_0's l2: 0.434138
[400]	valid_0's rmse: 0.663438	valid_0's l2: 0.44015
Early stopping, best iteration is:
[241]	valid_0's rmse: 0.657566	valid_0's l2: 0.432393

 Fold 0.6575656396154228

->-> Fold ran for 0 minutes 6 seconds

---- Fold 1 -----

5509 5509
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.796671	valid_0's l2: 0.634684
[200]	valid_0's rmse: 0.775128	valid_0's l2: 0.600823
[300]	valid_0's rmse: 0.774215	valid_0's l2: 0.599409
[400]	valid_0's rmse: 0.776819	valid_0's l2: 0.603448
Early stopping, best iteration is:
[252]	valid_0's rmse: 0.77344	valid_0's l2: 0.598209

 Fold 0.7734398473241461

->-> Fold ran for 0 minutes 8 seconds

---- Fold 2 -----

5509 5509
Training until validation scores don't improve f

In [204]:

print(f'\nOOF val score: {rmse(np.log(target), np.log(oofs))}')


OOF val score: 0.6709853824396907


In [205]:

vp0, tp0 = oofs, test_preds

In [206]:
len(test_preds)

1051

In [207]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import *
import time
training_start_time = time.time()

max_iter = 10
folds = StratifiedKFold(n_splits = max_iter)
oofs = np.zeros(len(train))
test_preds = np.zeros(len(test))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, pd.qcut(target, 10, labels=False, duplicates='drop'))):
    
    print(f'\n---- Fold {fold_} -----\n')
    
    fold_start_time = time.time()
    
    X_trn, y_trn = train.iloc[trn_idx][features], target.iloc[trn_idx]
    X_val, y_val = train.iloc[val_idx][features], target.iloc[val_idx]
    X_test = test[features]
    print(X_trn.shape[1], X_val.shape[1])
    
    
    clf = RandomForestRegressor()
    _ = clf.fit(X_trn, np.log(y_trn))

    oofs[val_idx] = np.exp(clf.predict(X_val))
    current_test_pred = np.exp(clf.predict(X_test))
    test_preds += np.exp(clf.predict(X_test))/max_iter
    
    
    print(f'\n Fold {rmse(np.log(y_val), np.log(oofs[val_idx]))}')
    
    fold_end_time = time.time()
    total_fold_time = int(fold_end_time - fold_start_time)
    
    print(f"\n->-> Fold ran for {(total_fold_time)//60} minutes {(total_fold_time)%60} seconds")
    

print(f'\nOOF val score: {rmse(np.log(target), np.log(oofs))}')
training_end_time = time.time()
total_training_time = int(training_end_time - training_start_time)

print(f'\n->-> Total training time: {(total_training_time)//60} minutes {(total_training_time)%60} seconds')


---- Fold 0 -----

5509 5509

 Fold 0.652354629622635

->-> Fold ran for 0 minutes 52 seconds

---- Fold 1 -----

5509 5509

 Fold 0.7804605818943127

->-> Fold ran for 0 minutes 54 seconds

---- Fold 2 -----

5509 5509

 Fold 0.6327692920681441

->-> Fold ran for 0 minutes 49 seconds

---- Fold 3 -----

5509 5509

 Fold 0.594813288319954

->-> Fold ran for 0 minutes 52 seconds

---- Fold 4 -----

5509 5509

 Fold 0.7153909776905518

->-> Fold ran for 0 minutes 49 seconds

---- Fold 5 -----

5509 5509

 Fold 0.5770260339154542

->-> Fold ran for 0 minutes 49 seconds

---- Fold 6 -----

5509 5509

 Fold 0.6498607517267875

->-> Fold ran for 0 minutes 48 seconds

---- Fold 7 -----

5509 5509

 Fold 0.6548987975015134

->-> Fold ran for 0 minutes 49 seconds

---- Fold 8 -----

5509 5509

 Fold 0.6944184047922788

->-> Fold ran for 0 minutes 49 seconds

---- Fold 9 -----

5509 5509

 Fold 0.6658436146309256

->-> Fold ran for 0 minutes 47 seconds

OOF val score: 0.6641556173868626

->-> T

In [162]:

vp1, tp1 = oofs, test_preds

In [168]:
training_start_time = time.time()
from xgboost import XGBRegressor
max_iter = 10
folds = StratifiedKFold(n_splits = max_iter)
oofs = np.zeros(len(train))
test_preds = np.zeros(len(test))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, pd.qcut(target, 10, labels=False, duplicates='drop'))):
    
    print(f'\n---- Fold {fold_} -----\n')
    
    fold_start_time = time.time()
    
    X_trn, y_trn = train.iloc[trn_idx][features], target.iloc[trn_idx]
    X_val, y_val = train.iloc[val_idx][features], target.iloc[val_idx]
    X_test = test[features]
    print(X_trn.shape[1], X_val.shape[1])
    

    clf = XGBRegressor(n_estimators=1000, max_depth=12, learning_rate=0.05, colsample_bytree=0.45)
    _ = clf.fit(X_trn.values, np.log(y_trn.values), eval_set = [(X_val.values, np.log(y_val.values))], verbose=100, early_stopping_rounds=200, eval_metric='rmse')
    

    oofs[val_idx] = np.exp(clf.predict(X_val.values))
    current_test_pred = np.exp(clf.predict(X_test.values))
    test_preds += np.exp(clf.predict(X_test.values))/max_iter
    
    
    print(f'\n Fold {rmse(np.log(y_val), np.log(oofs[val_idx]))}')
    
    fold_end_time = time.time()
    total_fold_time = int(fold_end_time - fold_start_time)
    
    print(f"\n->-> Fold ran for {(total_fold_time)//60} minutes {(total_fold_time)%60} seconds")
    

print(f'\nOOF val score: {rmse(np.log(target), np.log(oofs))}')
training_end_time = time.time()
total_training_time = int(training_end_time - training_start_time)

print(f'\n->-> Total training time: {(total_training_time)//60} minutes {(total_training_time)%60} seconds')


---- Fold 0 -----

5459 5459
[23:34:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:6.01262
Will train until validation_0-rmse hasn't improved in 200 rounds.
[100]	validation_0-rmse:0.665182
[200]	validation_0-rmse:0.660155
[300]	validation_0-rmse:0.654183
[400]	validation_0-rmse:0.650662
[500]	validation_0-rmse:0.647087
[600]	validation_0-rmse:0.646237
[700]	validation_0-rmse:0.644396
[800]	validation_0-rmse:0.64317
[900]	validation_0-rmse:0.642026
[999]	validation_0-rmse:0.641101

 Fold 0.6409051579324553

->-> Fold ran for 5 minutes 36 seconds

---- Fold 1 -----

5459 5459
[23:40:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:5.95724
Will train until validation_0-rmse hasn't improved in 200 rounds.
[100]	validation_0-rmse:0.758832
[200]	validation_0-rmse:0.750957
[300]	validation_0-rmse:0.74706
[

In [170]:

vp2, tp2 = oofs, test_preds

In [171]:
test_preds = tp1*0.4 + tp2*0.6

In [172]:
test_preds

array([2644.64885378,  417.68404691,  735.03588156, ..., 1455.74077124,
        516.80108871, 1463.5806302 ])

In [173]:
k=pd.DataFrame(test_preds,columns=["Selling_Price"])

In [174]:
k

,Selling_Price
0,2644.648854
1,417.684047
2,735.035882
3,373.429362
4,703.680464
...,...
1046,1188.159043
1047,352.627926
1048,1455.740771
1049,516.801089


In [175]:
k.to_excel("g5.xlsx",index=False)